In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_wines = pd.read_json('/content/drive/MyDrive/tobigs14_conference/make_survey/Meta_with_cluster.json')

food_list = [
    'Poultry',  # 4 양계류
    'Pork',  # -> 2,4 (바디감에 따라 높으면 4번, 낮으면 2번)
    'Beef', # -> 2,4 (바디감 1번, 산미  4번) if 바디감도 높고 산미도 높다면?
    'Lamb', # 4 양

    'Rich fish (salmon, tuna etc)', # 생선 2
    'Shellfish', # 생선 2
    'Lean fish',  # 생선 -> 2,4 (바디감에 따라 높으면 4번, 낮으면 2번)

    'Spicy food', # -> 4
    'Pasta',  # -> 2,4 (바디감에 따라 높으면 4번, 낮으면 2번)
    'Mushrooms', # -> 4
    'Vegetarian', # -> 2

    'Aperitif' # 식전주 -> 2번 클러스터
    'Appetizers and snacks',  # 4번 클러스터

    'Sweet desserts', # 달달한 디저트  -> 4 -> 7번 와인
    'Fruity desserts', # 과일 디저트 -> 4 -> 7번 와인

    'Mild and soft cheese', # -> 2
    'Mature and hard cheese',#  -> 1
    'Blue cheese', # -> 4
]

def get_recommendation(mood, food, sweet, tannin, body, acidity, df_wines=df_wines):
    # inputs = [mood, ]
    # mood (1~6), food ('Pork', 'Lamb' etc), sweet, tannin, body, acidity
    if mood in [1, 2]:
        if food in ['Poultry', 'Lamb']:
            candidate = df_wines.loc[df_wines.cluster==1,:]

        elif food in ['Pork', 'Lean fish', 'Pasta']:
            if body > 3:
                candidate = df_wines.loc[df_wines.cluster==4,:]
            else:
                candidate = df_wines.loc[df_wines.cluster==2,:]

        elif food == 'Beef':
            if body > acidity:
                candidate = df_wines.loc[df_wines.cluster==1,:]
            elif body < acidity:
                candidate = df_wines.loc[df_wines.cluster==4,:]
            else:
                candidate = df_wines.loc[df_wines.cluster.isin([1,4]),:]
        

        elif food in ['Rich fish (salmon, tuna etc)', 'Shellfish', 'Vegetarian', 'Aperitif', 'Mild and soft cheese']:
            candidate = df_wines.loc[df_wines.cluster==2,:]
        
        elif food in ['Spicy food', 'Mushrooms', 'Appetizers and snacks', 'Blue cheese']: 
            candidate = df_wines.loc[df_wines.cluster==4,:]

        elif food in ['Sweet desserts', 'Fruity desserts']:
            candidate = df_wines.loc[(df_wines.cluster==4) & (df_wines.type_id==7),:]

        elif food == 'Mature and hard cheese':
            candidate = df_wines.loc[df_wines.cluster==1,:]

        candidate['taste_score'] = sweet*candidate.sweetness + tannin*candidate.tannin + body*candidate.body + acidity*candidate.acidity
        candidate['taste_score'].fillna(candidate.taste_score.mean(), inplace=True)
        candidate.food.fillna({i: [] for i in candidate.index}, inplace=True)
        candidate.taste_score = candidate.taste_score + candidate.food.map(lambda foods:food in foods).astype(int) * 10
        
        candidate = candidate.sort_values(['taste_score'], ascending=False)[:30]
        candidate = candidate.sort_values(['rating_average', 'review_count'], ascending=False)

        wine_ids = list(candidate['wine_id'][:10])
        return wine_ids
    
    else:
        if mood == 3: #달콤한 분위기
            if food in ['Beef', 'Lamb', 'Pasta', 'Poultry', 'Pork', 'Lean fish']:
                candidate = df_wines.loc[df_wines.cluster==3,:]
                candidate=candidate.loc[candidate.food.map(lambda foods : food in foods)]

            else:
                candidate = df_wines.loc[df_wines.cluster==3,:]

        elif mood == 4: # 디저트
            if food in 'Sweet desserts':
                candidate = df_wines.loc[(df_wines.cluster==4) & (df_wines.type_id==7),:] 
                candidate=candidate.loc[candidate.food.map(lambda foods : 'Sweet desserts' in foods)]

            elif food in 'Fruity desserts':
                candidate = df_wines.loc[(df_wines.cluster==4) & (df_wines.type_id==7),:] 
                candidate=candidate.loc[candidate.food.map(lambda foods : 'Fruity desserts' in foods)]
                        
            else:
                candidate = df_wines.loc[df_wines.cluster==4,:]
        
        elif mood == 5: # 파티용
            candidate = df_wines.loc[(df_wines.cluster==2) & (df_wines.type_id==3),:] 
            food_includes = candidate.food.map(lambda foods : food in foods)
            if food_includes.sum() > 10:
                candidate=candidate.loc[food_includes]

        elif mood == 6:
            if food in ['Beef', 'Pork', 'Lamb', 'Poultry', 'Rich fish (salmon, tuna etc)', 'Shellfish' 'Spicy food', 'Vegetarian', 'Sweet desserts', 'Mild and soft cheese', 'Mature and hard cheese', 'Blue cheese']:
                candidate = df_wines.loc[(df_wines.cluster==4) & (df_wines.type_id==24),:]
            else:
                candidate = df_wines.loc[df_wines.type_id==24,:]
            
        
        candidate['taste_score'] = sweet*candidate.sweetness + tannin*candidate.tannin + body*candidate.body + acidity*candidate.acidity
        candidate['taste_score'].fillna(candidate.taste_score.mean(), inplace=True)
        candidate.food.fillna({i: [] for i in candidate.index}, inplace=True)
        candidate.taste_score = candidate.taste_score + candidate.food.map(lambda foods:food in foods).astype(int) * 5
        
        candidate = candidate.sort_values(['taste_score'], ascending=False)[:30]
        candidate = candidate.sort_values(['rating_average', 'review_count'], ascending=False)

        wine_ids = list(candidate['wine_id'][:10])
        return wine_ids
    
result = get_recommendation(1, 'Pork', 3, 2, 3, 5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret